In [ ]:
### Assignment 2 - Scraper Script

In [16]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re

In [ ]:
#product_name, product_review, and user_rating

In [142]:
# get all the page suffix
page_lst = []
main_page = requests.get('https://www.beeradvocate.com/beer/top-rated/')

soup = BeautifulSoup(main_page.text, 'html.parser')

pattern = "/beer/profile/(.+?)/(.+?)/"

pages_link = soup.findAll('a',attrs={'href': re.compile(pattern)})

pages_data = []

for l in pages_link:
    l = str(l)
    m = re.search(pattern,l)
    pages_data.append(m.group(1)+'/'+m.group(2))

In [54]:
pages_data[0]

'23222/78820'

In [139]:
# Scraping product_name, product_review, and user_rating
product_name = []
data = pd.DataFrame()

for p in pages_data:
    
    review = []
    rating = []
    
    page = requests.get('https://www.beeradvocate.com/beer/profile/'+p+'/')
    soup = BeautifulSoup(page.text,'html.parser')
    
    # product_names
    name = soup.find('div',class_ ="titleBar") 
    search = re.search('<h1>(.+?)<br/>',str(name))
    name = search.group(1)
    product_name.append(name)
    
    # progress checker
    print(name,len(product_name))
    
    
    comments = soup.findAll('div', attrs = {'id':'rating_fullview_content_2'})
    
    sleep(randint(2,8))
    
    for c in comments:
        
        # comments
        com = c.find('div')
        if com != None:
            cc = re.search('\">(.*?)</div>',str(com),flags = re.S)
            cc1 = str(cc.group(1)).replace('<br>','').replace('</br>','')
            review.append(cc1)
        else:
            review.append('')
            
        # ratings
        score = c.find('span', attrs = {'class':'BAscore_norm'})
        
        if score == None:
            score = c.find('span', attrs = {'class':'muted'})
            s = re.search('Rated:(.+?)by',str(score))
            s = str(s.group(1)).replace('<b>','').replace('</b>','')
            rating.append(float(s))
        else:
            rating.append(float(score.text))
    
    names = [name for i in range(len(rating))]
    
    d1 = pd.DataFrame({'product_name':names,'product_review':review,'user_rating':rating})
    data = pd.concat([data,d1])


data.reset_index(drop = True, inplace = True)

Kentucky Brunch Brand Stout 1
Vanilla Bean Assassin 2
Marshmallow Handjee 3
Abraxas - Barrel-Aged 4
Hunahpu's Imperial Stout - Double Barrel Aged 5
Blessed 6
Heady Topper 7
O.W.K. 8
Mornin' Delight 9
King Julius 10
A Deal With The Devil - Triple Oak-Aged 11
King JJJuliusss 12
Zenne Y Frontera 13
Pliny The Younger 14
Coconut Vibes - Barrel-Aged 15
Barrel Aged Imperial German Chocolate Cupcake Stout 16
Fundamental Observation 17
Very Hazy 18
Bourbon County Brand Coffee Stout 19
V.S.O.J. 20
Oude Fermier 21
A Deal With The Devil - Double Oak-Aged 22
Clover 23
Anabasis 24
Chemtrailmix - Rye Barrel 25
Double Barrel V.S.O.J. 26
Julius 27
Fou' Foune 28
Monster Tones 29
Very Green 30
Assassin 31
Juice Machine 32
Very GGGreennn 33
Westly 34
It Was All A Dream 35
Pliny The Elder 36
CBS (Canadian Breakfast Stout) 37
Samuel 38
Abner 39
Framboise Du Fermier 40
Double Sunshine 41
JJJuliusss! 42
Speedway Stout - Vietnamese Coffee - Bourbon-Barrel Aged 43
Dinner 44
Duck Duck Gooze 45
Lou Pepe - Kriek 4

In [140]:
data

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,,4.75
1,Kentucky Brunch Brand Stout,"Amazing stout, expensive but worth the price!",5.00
2,Kentucky Brunch Brand Stout,,4.52
3,Kentucky Brunch Brand Stout,,5.00
4,Kentucky Brunch Brand Stout,This beer meet and exceeded all the unreal hyp...,5.00
...,...,...,...
4995,Samuel Adams Utopias,,4.88
4996,Samuel Adams Utopias,,4.14
4997,Samuel Adams Utopias,,4.56
4998,Samuel Adams Utopias,"Not sure it's worth the price tag, but the 202...",4.59


In [141]:
data.to_csv('beer_reviews.csv')